In [1]:
# Paquetes
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import dateutil.relativedelta
import gc

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',400)

In [2]:
import s3fs
import pyarrow.parquet as pq

def read_parquet(ruta:str): 
    fs = s3fs.S3FileSystem()
    dataset = pq.ParquetDataset(ruta, filesystem=fs)
    table = dataset.read()
    df = table.to_pandas()
    return df

In [3]:
#Atento

path_atento = 's3://adl-refined-dev-popular/data_orig/tmp/tmpg_productos_activo-tarjeta-credito-leads_atento_H20200901-20201001.csv'

#Project

path_project_int    = 's3://adl-refined-dev-popular/data_orig/tmp/tmpg_devolucion_interesados_20201005_project.csv'
path_project_perf_a = 's3://adl-refined-dev-popular/data_orig/tmp/tmpg_devolucion_perfilado_activo_20201005_project.csv'
path_project_perf_i = 's3://adl-refined-dev-popular/data_orig/tmp/tmpg_devolucion_perfilado_inactivo_20201005_project.csv'
path_project_prea   = 's3://adl-refined-dev-popular/data_orig/tmp/tmpg_devolucion_preaprobados_20201005_project.csv'

#Nexa

path_nexa_sept = 's3://adl-refined-dev-popular/data_orig/tmp/tmpg_BP_VENTAS_TC_TRADICIONAL_BASE_Septiembre_003.csv'
path_nexa_octu = 's3://adl-refined-dev-popular/data_orig/tmp/tmpg_BP_VENTAS_TC_TRADICIONAL_BASE_Octubre_001.csv'

In [4]:
data_atento = pd.read_csv(path_atento,sep='|', encoding='latin-1',dtype={'ID_CLI':str})

data_project_int    = pd.read_csv(path_project_int,sep='|', encoding='latin-1', dtype={'ID_PRINCIPAL':str})
data_project_perf_a = pd.read_csv(path_project_perf_a,sep='|', encoding='latin-1', dtype={'ID_PRINCIPAL':str})
data_project_perf_i = pd.read_csv(path_project_perf_i,sep='|', encoding='latin-1', dtype={'ID_PRINCIPAL':str})
data_project_prea   = pd.read_csv(path_project_prea,sep='|', encoding='latin-1', dtype={'ID_PRINCIPAL':str})

data_nexa_sept = pd.read_csv(path_nexa_sept,sep='|', encoding='latin-1', dtype={'IDPRINCIPAL':str})
data_nexa_octu = pd.read_csv(path_nexa_octu,sep='|', encoding='latin-1', dtype={'IDPRINCIPAL':str})

/home/jupyter/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data_project = data_project_int.append(data_project_perf_a).append(data_project_perf_i).append(data_project_prea)

data_nexa_sept["FECHA INICIO LLAMADA"] = pd.to_datetime(data_nexa_sept["FECHA INICIO LLAMADA"], format="%d/%m/%Y %H:%M")
data_nexa_sept["FECHA FINAL LLAMADA"]  = pd.to_datetime(data_nexa_sept["FECHA INICIO LLAMADA"], format="%d/%m/%Y %H:%M")

data_project["FECHA INICIO LLAMADA"] = pd.to_datetime(data_project["FECHA INICIO LLAMADA"], format="%d/%m/%Y")
data_project["FECHA FINAL LLAMADA"]  = pd.to_datetime(data_project["FECHA FINAL LLAMADA"], format="%d/%m/%Y")

data_atento["FECHA_INCIO_LLAMADA"] = pd.to_datetime(data_atento["FECHA_INCIO_LLAMADA"], format="%Y-%m-%d")
data_atento["FECHA_FINAL_LLAMADA"] = pd.to_datetime(data_atento["FECHA_FINAL_LLAMADA"], format="%Y-%m-%d")

In [6]:
data_nexa    = data_nexa_sept[(data_nexa_sept["FECHA INICIO LLAMADA"] >= '2020-09-01') | (data_nexa_sept["FECHA INICIO LLAMADA"] <= '2020-09-30')]

data_project = data_project[(data_project['FECHA INICIO LLAMADA'] >= "2020-09-01") & (data_project['FECHA INICIO LLAMADA'] <= "2020-09-30")]

data_atento  = data_atento[(data_atento["FECHA_INCIO_LLAMADA"] >= '2020-09-01') & (data_atento["FECHA_INCIO_LLAMADA"] <= '2020-09-30')]

In [7]:
data_atento['ID_CLI'] = data_atento['ID_CLI'].astype(int)

data_project['ID_PRINCIPAL'] = data_project['ID_PRINCIPAL'].astype(int)

data_nexa['IDPRINCIPAL'] = data_nexa['IDPRINCIPAL'].astype(int)

In [8]:
data_atento  = data_atento[["ID_CLI","CAMPAÑA","ESTRUCTURA","TIPIFICACIÓN_1","TIPIFICACIÓN_2"]]

data_project = data_project[["ID_PRINCIPAL","CAMPAÑA","ESTRUCTURA","TIPIFICACIÓN 1","TIPIFICACIÓN 2"]]

data_nexa    = data_nexa[["IDPRINCIPAL","CAMPAÑA","ESTRUCTURA","TIPIFICACION_NIVEL_2","TIPIFICACION_NIVEL_3"]]

In [9]:
#data_atento = data_atento[data_atento['CAMPAÑA']=='Banco Popular Perfilado Activo']
data_atento.columns = ['CEDULA', 'CAMPAÑA', 'ESTRUCTURA', 'TIPIFICACIÓN_1', 'TIPIFICACIÓN_2']

data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("AUT - Anunciadora", "NO CONTACTO")
data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("AUT - HangUp", "NO CONTACTO")
data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("AUT - Silencio", "NO CONTACTO")
data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("AUT - Mensaje de la Operadora", "NO CONTACTO")
data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("AUT - Número Inválido", "NO CONTACTO")
data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("AUT - No contesta", "NO CONTACTO")
data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("AUT - Fax", "NO CONTACTO")
data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("AUT - Ocupado", "NO CONTACTO")
data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("AUT - Sin Tabulación", "NO CONTACTO")
data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("AUT - Impossible Completar", "NO CONTACTO")

data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("VOLVER A LLAMAR A SOLICITUD DEL CLIENTE", "VOLVER A LLAMAR")
data_atento["TIPIFICACIÓN_1"] = data_atento["TIPIFICACIÓN_1"].replace("CLIENTE NO GESTIONABLE", "NO EFECTIVO")

In [10]:
data_project.columns = ['CEDULA', 'CAMPAÑA', 'ESTRUCTURA', 'TIPIFICACIÓN_1', 'TIPIFICACIÓN_2']
data_project['TIPIFICACIÓN_2'] = data_project['TIPIFICACIÓN_1']

In [11]:
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_2'].replace('No desea mas deudas LD/TDC', 'NO INTERESADO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('En Gestión', 'NO APLICA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Teléfono equivocado', 'NO EFECTIVO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No contesta', 'NO CONTACTO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Linea ocupada', 'NO CONTACTO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Buzón de Voz', 'NO CONTACTO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Mensaje con tercero', 'NO EFECTIVO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No brinda informacion', 'NO INTERESADO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Preselecta no viable - Política', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Cliente se dirige a la oficina', 'NO EFECTIVO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No Cumple Perfil - Ingresos', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Cuelga la llamada', 'NO EFECTIVO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Molesto por llamadas frecuentes', 'NO INTERESADO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Mala experiencia con el Banco', 'NO INTERESADO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Interesado más adelante', 'VOLVER A LLAMAR')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No Cumple Perfil - Actividad Eco', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No autoriza tratamiento de datos', 'NO EFECTIVO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No se lo permite un tercero', 'NO INTERESADO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Crédito en tramite', 'NO APLICA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Bien atendido por otra entidad', 'NO INTERESADO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No Cumple Perfil - Cliente repor', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Desea mayor monto', 'NO INTERESADO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No Cumple Perfil - Antigüedad La', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Cliente fuera de la ciudad- Pais', 'NO EFECTIVO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Venta Efectiva - Visa', 'VENTA EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Preselecta no viable - Calificac', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Venta Cantada - Libre Destino', 'VENTA EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Incapacidad fisica', 'NO APLICA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Tasa de interes', 'NO INTERESADO')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Cuota de manejo alta', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No Cumple Perfil - No Cuenta Con', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Venta Efectiva - Duo', 'VENTA EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Fallecido', 'NO APLICA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Cliente No cumple edad', 'NO APLICA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Cobertura geografica', 'NO APLICA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No Cumple Perfil - Actividad Económica', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No Cumple Perfil - Cliente reportado', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No Cumple Perfil - Antigüedad Laboral', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('No Cumple Perfil - No Cuenta Con Documentos', 'VENTA NO EFECTIVA')
data_project['TIPIFICACIÓN_1'] = data_project['TIPIFICACIÓN_1'].replace('Preselecta no viable - Calificacion Credito', 'VENTA NO EFECTIVA')

In [12]:
data_nexa.columns = ['CEDULA', 'CAMPAÑA', 'ESTRUCTURA', 'TIPIFICACIÓN_1', 'TIPIFICACIÓN_2']

In [13]:
data_atento['Vendors']  = 'Atento'
data_project['Vendors'] = 'Project'
data_nexa['Vendors']    = 'Nexa'

In [15]:
df_final = data_atento.append(data_project).append(data_nexa)

df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].str.lower()

In [16]:
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].str.replace(u"á", "a") 
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].str.replace(u"é", "e")
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].str.replace(u"í", "i")
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].str.replace(u"ó", "o")
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].str.replace(u"ú", "u")

In [17]:
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('adl - no aprobado', 'adl  no aprobado (polticas  no viable)')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('adl  ya tiene producto aprobado', 'adl - ya tiene producto aprobado')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('bien atendido por otra entidad', 'credito reciente otro banco')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('cliente fuera de la ciudad', 'cliente fuera de la ciudad- pais')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('cliente fuera del pais', 'cliente fuera de la ciudad- pais')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('cliente no cumple edad', 'no cumple edad')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('cuelga la llamada', 'no brinda informacion o cuelga llamada')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('no brinda informacion', 'no brinda informacion o cuelga llamada')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('no cumple perfil actividad economica', 'no cumple perfil - actividad economica')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('no cumple perfil antiguedad laboral', 'no cumple perfil - antigüedad laboral')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('no cumple perfil cliente reportado', 'no cumple perfil - cliente reportado')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('no cumple perfil ingresos', 'no cumple perfil - ingresos')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('no cumple perfil no cuenta con documentos', 'no cumple perfil - no cuenta con documentos')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('no cumple perfil, cliente reportado', 'no cumple perfil - cliente reportado')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('no cumple perfil, ingresos', 'no cumple perfil - ingresos')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('no cumple perfil, politica', 'no cumple perfil - politica')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('no desea mas deudas ld/tdc', 'no desea mas deudas')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('preselecta no viable capacidad de pago', 'preselecta no viable - capacidad de pago')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('preselecta no viable habitos', 'preselecta no viable - habitos')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('preselecta no viable politica', 'preselecta no viable - politica')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('preselecta no viable score', 'preselecta no viable . score')
df_final["TIPIFICACIÓN_2"] = df_final["TIPIFICACIÓN_2"].replace('visa', 'venta efectiva - visa')

In [18]:
df_final["TIPIFICACIÓN_1"].value_counts()

NO CONTACTO                     147207
NO INTERESADO                    11174
VOLVER A LLAMAR                   4283
NO EFECTIVO                       4215
VENTA EFECTIVA                    1561
FALLAS                            1066
NO APLICA                          539
VENTA NO EFECTIVA                  466
FALLAS                              24
ADL Sim Card con novedad             5
ADL por riesgo                       2
AGENDAMIENTO                         2
Venta Efectiva - Master Card         1
Name: TIPIFICACIÓN_1, dtype: int64

In [19]:
df_final["TIPIFICACIÓN_2"].value_counts()

no desea mas deudas                            5641
no contesta                                    4204
interesado mas adelante                        4063
no brinda informacion o cuelga llamada         2972
mensaje con tercero                            2240
telefono equivocado                            1309
venta efectiva - visa                          1275
no le gustan las tdc                           1194
mensaje contestador                             957
llamada caida                                   660
ya tiene otra tarjeta                           540
linea inhabilitada                              507
llamada muda                                    376
ilocalizado                                     364
linea ocupada                                   330
molesto por llamadas frecuentes                 326
credito en tramite                              290
interesado cliente se dirige a la oficina       201
buzon de voz                                    200
duo         

In [26]:
df_perfilados = read_parquet('s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_PilotoPerfVars')

In [27]:
df_perfilados = df_perfilados[(df_perfilados.grupo_pilot != 'grupo_mkd_perf_alta') & (df_perfilados.grupo_pilot != 'grupo_mkd_perf_baja')]

In [28]:
df_perfilados.numero_identificacion = df_perfilados.numero_identificacion.apply(str)
df_perfilados.numero_identificacion = df_perfilados.numero_identificacion.apply(float)
df_perfilados.numero_identificacion = df_perfilados.numero_identificacion.apply(int)

df_final.CEDULA = df_final.CEDULA.apply(str)
df_final.CEDULA = df_final.CEDULA.apply(float)
df_final.CEDULA = df_final.CEDULA.apply(int)

In [38]:
df_vars_tip = pd.merge(df_perfilados, df_final, left_on = 'numero_identificacion', right_on = 'CEDULA', how = 'left')

In [39]:
df_vars_tip[df_vars_tip.CAMPAÑA.isna() == True]

,tipo_identificacion,numero_identificacion,grupo_prob,segmento,score,decil,grupo_pilot,sa21s,re20s,tel20s,sum_tx_linea_pasv,at35af,num_tx_ult_1mes_ahr,all255,pasv_dias_desde_ultima_trans,ul27s,at20sf,rt20s,at28af,bc02s,dm001,br01s,agg214,CEDULA,CAMPAÑA,ESTRUCTURA,TIPIFICACIÓN_1,TIPIFICACIÓN_2,Vendors
6,1,137820,grupo_prob_baja,seg_perfilado,0.000002,5.0,grupo_control_perf_baja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,1,230304,grupo_prob_alta,seg_perfilado,0.000163,3.0,grupo_tratm_perf_alta,37.0,171.0,120.0,21.0,18307.0,8.000000,0.0,8.0,4.0,187.0,73.0,112127.0,3.0,64.0,17.0,149225.0,NaN,NaN,NaN,NaN,NaN,NaN
35,1,239477,grupo_prob_baja,seg_perfilado,0.000033,4.0,grupo_control_perf_baja,46.0,NaN,119.0,NaN,NaN,NaN,105.0,131.0,1.0,167.0,NaN,NaN,NaN,85.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,1,241685,grupo_prob_baja,seg_perfilado,0.000003,5.0,grupo_control_perf_baja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,1,241685,grupo_prob_baja,seg_perfilado,0.000003,5.0,grupo_control_perf_baja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10056,1,80171155,grupo_prob_baja,seg_perfilado,0.000017,5.0,grupo_control_perf_baja,5.0,126.0,62.0,NaN,572.0,0.000000,324.0,37.0,3.0,126.0,42.0,3179.0,2.0,39.0,8.0,5096.0,NaN,NaN,NaN,NaN,NaN,NaN
10057,1,80171910,grupo_prob_alta,seg_perfilado,0.000276,2.0,grupo_control_perf_alta,47.0,10.0,186.0,16.0,1004.0,14.000000,0.0,7.0,0.0,123.0,62.0,7361.0,3.0,38.0,3.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN
10058,1,80172168,grupo_prob_baja,seg_perfilado,0.000067,4.0,grupo_control_perf_baja,NaN,191.0,203.0,16.0,1432.0,21.687500,0.0,8.0,5.0,191.0,NaN,6860.0,2.0,38.0,5.0,14965.0,NaN,NaN,NaN,NaN,NaN,NaN
10061,1,80176061,grupo_prob_baja,seg_perfilado,0.000071,4.0,grupo_control_perf_baja,23.0,75.0,124.0,6.0,17784.0,9.333333,1134.0,11.0,5.0,118.0,75.0,58495.0,NaN,36.0,0.0,27495.0,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df_vars_tip.TIPIFICACIÓN_1.value_counts()

NO CONTACTO          4024
NO INTERESADO        1884
NO EFECTIVO           903
VOLVER A LLAMAR       253
VENTA NO EFECTIVA     177
FALLAS                 71
NO APLICA              64
VENTA EFECTIVA          5
Name: TIPIFICACIÓN_1, dtype: int64

In [41]:
df_vars_tip.TIPIFICACIÓN_2.value_counts()

no desea mas deudas                            1371
no contesta                                     903
telefono equivocado                             594
no brinda informacion o cuelga llamada          316
linea ocupada                                   255
interesado mas adelante                         221
linea inhabilitada                              202
mensaje con tercero                             189
buzon de voz                                    133
mensaje contestador                              91
no le gustan las tdc                             67
ilocalizado                                      64
adl  no aprobado (polticas  no viable)           63
cliente se dirige a la oficina                   59
llamada caida                                    49
no cumple perfil - ingresos                      44
ya tiene otra tarjeta                            41
mala experiencia con el banco                    39
preselecta no viable - politica                  34
desempleado 

In [25]:
df_vars_tip.to_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_tipificaciones_sept")

In [33]:
df_final2 = df_final[['CEDULA','CAMPAÑA']].drop_duplicates()

In [ ]:
df_final.CEDULA.nunique()

In [ ]:
data_atento['FECHA_INCIO_LLAMADA'] = data_atento['FECHA_INCIO_LLAMADA'].str.slice(0, 10)
data_atento['FECHA_FINAL_LLAMADA'] = data_atento['FECHA_FINAL_LLAMADA'].str.slice(0, 10)

In [ ]:
data_atento["FECHA_INCIO_LLAMADA"] = pd.to_datetime(data_atento["FECHA_INCIO_LLAMADA"], format="%Y-%m-%d")
data_atento["FECHA_FINAL_LLAMADA"] = pd.to_datetime(data_atento["FECHA_FINAL_LLAMADA"], format="%Y-%m-%d")

# data_atento["FECHA_INCIO_LLAMADA"] = pd.to_datetime(data_atento["FECHA_INCIO_LLAMADA"], format="%Y-%m-%d %H:%M:%S")
# data_atento["FECHA_FINAL_LLAMADA"] = pd.to_datetime(data_atento["FECHA_FINAL_LLAMADA"], format="%Y-%m-%d %H:%M:%S")

In [ ]:
data_atento["FECHA_INCIO_LLAMADA"].min()

In [ ]:
data_atento["FECHA_INCIO_LLAMADA"].max()

In [ ]:
data_atento = data_atento.drop_duplicates()

In [ ]:
data_atento[data_atento['ID_CLI']==32310868]

In [ ]:
data_atento[(data_atento.CAMPAÑA == 'Banco Popular Perfilado Activo') & (data_atento.TIPIFICACIÓN_1 == 'VENTA EFECTIVA')]

In [ ]:
prueba = data_atento.ID_CLI.value_counts().to_frame()

In [ ]:
prueba[prueba.ID_CLI == 2]

In [ ]:
prueba.ID_CLI.value_counts()